In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras
from getpass import getpass

In [56]:
from IndyData.ACS.fns import get_ACS
from IndyData.ACS.variables import population_variables

In [57]:
def create_acs_population_table(con, names, table_name='acs_population'):
    
    cur = conn.cursor()
    columns = [i + ' integer' for i in names]
    columns = ', '.join(columns)
    query  = f'CREATE TABLE {table_name} (GEOID BIGINT, county_name varchar, tract_name varchar, year integer, CONSTRAINT pk_GEOID_YEAR PRIMARY KEY (GEOID,year), %s);' % columns
    
    try:
        cur.execute(query)
    except Exception as e:
        print(e)
        print("Error in creating ACS Population Table")

    conn.commit() 
    conn.close()
    cur.close()
    print(f'Successfully created table: {table_name}')
    

# Create ACS Population Table

In [58]:
names = [i for i in population_variables.keys()]

In [59]:
try:
    conn = psycopg2.connect(
        database = "indydata_testing", 
        user = "vinniepalazeti", 
        password = getpass(), 
        host = "localhost", 
        port = "5432")

except:
    print("No connection to Database") 
    
create_acs_population_table(conn, names)

 ········


Successfully created table: acs_population


# Pull ACS Data

In [33]:
df_list = []
for year in range(2010,2020):

    acs_data = get_ACS(
        population_variables, 
        year=year
    )
    df_list.append(acs_data)
    
acs_data = pd.concat(df_list)

In [34]:
acs_data = acs_data.reset_index()

In [62]:
acs_data.head()

,GEOID,Population_Total,Population_White,Population_Black,Population_Arab,Population_Asian,Population_Hispanic,Population_American_Indiana_and_Alaska_Native,Population_Hawaiian_and_Pacific_Islander,Population_Non_Hispanic,...,Population_Female_62_64,Population_Female_65_66,Population_Female_67_69,Population_Female_70_74,Population_Female_75_79,Population_Female_80_84,Population_Female_85_plus,tract_name,county_name,year
0,18001030100,4617,4415,49,0,12,177,0,0,4440,...,70,46,43,77,39,79,34,Census Tract 301,Adams County,2010
1,18001030200,4972,4676,11,0,9,453,60,0,4519,...,76,76,144,47,80,83,35,Census Tract 302,Adams County,2010
2,18001030300,5752,5454,0,0,0,559,0,0,5193,...,45,0,21,153,92,161,136,Census Tract 303,Adams County,2010
3,18001030400,3261,3211,9,0,0,41,0,0,3220,...,25,26,29,69,35,14,61,Census Tract 304,Adams County,2010
4,18001030500,6061,6052,0,0,3,8,0,0,6053,...,83,22,16,58,82,37,23,Census Tract 305,Adams County,2010


# Insert ACS Data to SQL

In [63]:
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [64]:
try:
    conn = psycopg2.connect(
        database = "indydata_testing", 
        user = "vinniepalazeti", 
        password = getpass(), 
        host = "localhost", 
        port = "5432")
except:
    print("No connection to Database") 
    
execute_values(conn, acs_data, 'acs_population')

 ········


execute_values() done


# Drop Table

In [52]:
table_name = 'acs_population'

try:
    conn = psycopg2.connect(
        database = "indydata_testing", 
        user = "vinniepalazeti", 
        password = getpass(), 
        host = "localhost", 
        port = "5432")
except:
    print("No connection to Database") 

cur = conn.cursor()

cur.execute(f'DROP TABLE {table_name};')  
conn.commit()
conn.close()

 ········
